In [17]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import TextVectorization
from transformers import AutoTokenizer

import numpy as np
import os
import requests
import io
import pandas as pd
import string


from tqdm import tqdm
from PIL import Image


config = tf.compat.v1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.95 # Change this value as per requirement
tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))

2023-09-23 18:48:19.891636: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-23 18:48:19.894432: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-23 18:48:19.895142: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-23 18:48:19.918307: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-23 18:48:19.918518: I tensorflow/core/co

In [3]:
image_shape = (100, 100, 3)


def download_image_buffer(url):
    img = None
    try:
        data = requests.get(url,headers={"user-agent": "I am a valid user, please give me image"}).content
        img = Image.open(io.BytesIO(data))
        img = img.resize(image_shape[:2])
        img = np.array(img, dtype=np.float64)/255
    except:
        pass
    
    return img


learning_rate = 0.001
weight_decay = 0.0001
batch_size = 32
num_epochs = 10
image_size = 100 
patch_size = 5 
num_patches = (image_size // patch_size) ** 2
projection_dim = 64
num_heads = 4
transformer_units = [
    projection_dim * 2,
    projection_dim,
]  # Size of the transformer layers
transformer_layers = 8
mlp_head_units = [2048, 1024]  # Size of the dense layers of the final classifier

In [4]:
sequence_length = 128
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
vocab_size = tokenizer.vocab_size

In [18]:
_max=100000

with open("./datasets/conceptual-12m/cc12m.tsv", "r", encoding="utf-8") as dataset_file:
    urls, texts = [], []

    for i, item in enumerate(tqdm(dataset_file, total=_max)):
        if i == _max:
            break
        url, text = item.split("\t")

        urls.append(url)
        texts.append(text)

urls = np.array(urls)
texts = np.array(texts)

  0%|                                                                                | 0/100000 [00:00<?, ?it/s]

100%|████████████████████████████████████████████████████████████████| 100000/100000 [00:01<00:00, 76815.62it/s]


In [8]:
def preprocess(url, text):
    try:
        url = url.numpy().decode('utf-8')
    except:
        pass

    try:
        text = text.numpy().decode('utf-8')
    except:
        pass

    img = download_image_buffer(url)

    # print(img is not None)
    
    if img is None:
        text = "A blank image"
        img = np.zeros(image_shape, dtype=np.float64)

    tokenized_text = np.array(tokenizer.encode(text))[:sequence_length]

    return img, tokenized_text

def dict_map(img, tokenized_text):
    return ({
        "image": img,
        "text": tokenized_text[:-1]
    }, tokenized_text[1:])

In [9]:
a, b = [], []

for u, t in tqdm(zip(urls[:20], texts[:20]), total=20):
    img, tokenized = preprocess(u, t)

    a.append(img)
    b.append(tokenized)


100%|███████████████████████████████████████████████████████████████████████████| 20/20 [00:10<00:00,  1.92it/s]


In [15]:
dataset = (tf.data.Dataset.from_tensor_slices((urls, texts))
           .shuffle(1000)
           .map(lambda url, text: tf.py_function(preprocess, [url, text], [tf.float64, tf.int32]))
        #    .map(dict_map)
           .batch(batch_size)
         #   .prefetch(tf.data.AUTOTUNE)
           )

In [16]:
dataset.as_numpy_iterator().next()

2023-09-23 17:29:54.831366: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [100000]
	 [[{{node Placeholder/_0}}]]
2023-09-23 17:29:54.831786: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype string and shape [100000]
	 [[{{node Placeholder/_1}}]]


InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Cannot add tensor to the batch: number of elements does not match. Shapes are: [tensor]: [5], [batch]: [8] [Op:IteratorGetNext]

In [ ]:
def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x

class Patches(layers.Layer):
    def __init__(self, patch_size, **kwargs):
        super().__init__(**kwargs)
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches

class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim, **kwargs):
        super().__init__(**kwargs)
        self.num_patches = num_patches
        self.projection = layers.Dense(units=projection_dim)
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded


class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "sequence_length": self.sequence_length,
                "vocab_size": self.vocab_size,
                "embed_dim": self.embed_dim,
            }
        )
        return config

In [ ]:
class EncoderBlock(layers.Layer):
    def __init__(self, num_heads, embedding_dims, hidden_layers, hidden_units="same", attn_dropout=0.2, ff_dropout=0.2, **kwargs):
        super().__init__(**kwargs)

        self.num_heads = num_heads
        self.embedding_dims = embedding_dims
        self.hidden_layers = hidden_layers
        self.hidden_units = hidden_units if hidden_units != "same" else embedding_dims
        self.attn = layers.MultiHeadAttention(
            num_heads=num_heads,
            key_dim=embedding_dims,
            dropout=attn_dropout
        )
        self.attn_norm = layers.LayerNormalization()
        self.ff = []

        for _ in range(hidden_layers):
            self.ff.append(layers.Dense(
                self.hidden_units,
                activation=tf.nn.gelu
            ))
            self.ff.append(layers.Dropout(ff_dropout))

        self.ff = keras.Sequential(self.ff)
        self.ff_norm = layers.LayerNormalization()
    
    def call(self, inputs, mask=None):
        attn_output = self.attn(
            query=inputs,
            key=inputs,
            value=inputs
        )

        attn_output_norm = self.attn_norm(inputs + attn_output)

        x = self.ff(attn_output_norm)
        
        ff_output = self.ff_norm(attn_output + x)

        return {
            "ff_output": ff_output,
            "attn_output": attn_output
        }

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "embedding_dim": self.embedding_dims,
                "num_heads": self.num_heads,
                "ff_hidden_layers": self.hidden_layers,
                "ff_hidden_units": self.hidden_units
            }
        )
        return config

In [ ]:
class DecoderBlock(layers.Layer):
    def __init__(self, 
                 num_heads_self, 
                 embedding_dims_self, 
                 hidden_layers, 
                 hidden_units="same", 
                 num_heads_cross="same", 
                 embedding_dims_cross="same", 
                 attn_dropout=0.2, 
                 ff_dropout=0.2, **kwargs):
        super().__init__(**kwargs)

        self.num_heads = num_heads
        self.embedding_dims = embedding_dims_self
        self.hidden_layers = hidden_layers
        self.hidden_units = hidden_units if hidden_units != "same" else embedding_dims_self
        self.num_heads_cross = num_heads_cross if num_heads_cross != "same" else num_heads_self
        self.embedding_dims_cross = embedding_dims_cross if embedding_dims_cross != "same" else embedding_dims_self

        self.self_attn = layers.MultiHeadAttention(
            num_heads=num_heads_self, 
            key_dim=embedding_dims_self,
            dropout=attn_dropout,
            name="Self-attn-decoder"
        )
        self.self_norm = layers.LayerNormalization()

        self.cross_attn = layers.MultiHeadAttention(
            num_heads=self.num_heads_cross, 
            key_dim=self.embedding_dims_cross,
            dropout=attn_dropout,
            name="cross-attn-decoder"
        )
        self.cross_norm = layers.LayerNormalization()
        
        self.ff = []

        for _ in range(hidden_layers):
            self.ff.append(layers.Dense(
                self.hidden_units,
                activation=tf.nn.gelu
            ))
            self.ff.append(layers.Dropout(ff_dropout))

        self.ff = keras.Sequential(self.ff)
        self.ff_norm = layers.LayerNormalization()
        self.add = layers.Add()  # instead of `+` to preserve mask
        self.supports_masking = True

    def call(self, inputs, encoder_outputs, mask=None):
        self_attn_output = self.self_attn(
            query=inputs, key=inputs, value=inputs, use_causal_mask=True
        )
        self_attn_output = self.self_norm(self.add([inputs, self_attn_output]))

        cross_attn_output = self.cross_attn(
            query=self_attn_output, key=encoder_outputs, value=encoder_outputs,
        )
        cross_attn_output = self.cross_norm(self.add([self_attn_output, cross_attn_output]))

        ff_output = self.ff(cross_attn_output)
        ff_output = self.ff_norm(self.add([ff_output, cross_attn_output]))

        return ff_output
    
    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "embedding_dim": self.embedding_dims,
                "embedding_dim_cross": self.embedding_dims_cross,
                "num_heads": self.num_heads,
                "num_heads_cross": self.num_heads_cross,
                "ff_hidden_layers": self.hidden_layers,
                "ff_hidden_units": self.hidden_units
            }
        )
        return config

In [35]:
num_blocks_encoder = 2
num_blocks_decoder = 2
num_heads = 8
embedding_dims = 64
hidden_layers = 2
sequence_length = 128
text_embedding_dims = 128

In [ ]:
image_input = layers.Input(shape=image_shape, name="image")
caption_input = layers.Input(shape=(sequence_length), name="text")

patches = Patches(patch_size)(image_input)
encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

encoded_captions = PositionalEmbedding(sequence_length, vocab_size, embedding_dims)(caption_input)

attn_weights = []

for _ in range(num_blocks_encoder):
    encoded_patches = EncoderBlock(
        num_heads,
        embedding_dims,
        hidden_layers
    )(encoded_patches)

    attn_weights.append(encoded_patches["attn_output"])
    encoded_patches = encoded_patches["ff_output"]


for _ in range(num_blocks_decoder):
    encoded_captions = DecoderBlock(
        num_heads,
        embedding_dims,
        hidden_layers
    )(encoded_captions, encoded_patches)

# top_layer = layers.Flatten()(encoded_captions)
top_layer = layers.Dense(1024, activation=tf.nn.gelu)(encoded_captions)
top_layer = layers.Dense(vocab_size, activation=tf.nn.softmax)(top_layer)

transformer = Model(inputs=[image_input, caption_input], outputs=top_layer, name="transformer")
transformer.compile(
    keras.optimizers.AdamW(),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)
transformer.summary()